# OCR Quality Assessment with `impresso-pipelines` Package

<a target="_blank" href="https://colab.research.google.com/github/impresso/impresso-datalab-notebooks/blob/main/annotate/ocrqa_pipeline_demo.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


### What is this notebook about?

This notebook demonstrates how to use the Impresso OCR Quality Assessment (ocrqa) pipeline to evaluate the quality of text recognition. It provides a simple example and an overview of advanced usage options.

#### Why is this useful?

- OCR (Optical Character Recognition) quality can vary significantly across different archives, historical fonts, time periods, and OCR systems.
- A quality score helps you quickly assess text readability, filter out low-quality texts, and compare OCR results from different sources.

#### How it works

The tool assigns a score between 0 and 1 based on sets of unique words, not individual word occurrences:

- 1.0 = Perfect quality (all words are recognized)
- 0.0 = Very poor quality (no words are recognized)
- Example: A score of 0.8 means 80% of unique words were recognized.

#### Technical background

- The `ocrqa` tool relies on lists of known words for each supported language.
- These word lists are compiled from sources such as Wikipedia articles and [Wortschatz Leipzig](https://wortschatz.uni-leipzig.de/en/download) sentences.
- The word lists are stored efficiently using "Bloom filters", which allow for fast and memory-efficient membership testing.
- The tool supports multiple languages (e.g., German, French, English), each with its
  own separate word list. However, frequent foreign words might be included in the list
  as well, according to their appearance in the source texts (e.g. Wikipedia articles).
- If the language of an input text is not specified, the tool uses Impresso's language identification tool to determine the language automatically.
- If a language is not supported, the tool will notify you accordingly.


## What will you learn?

In this notebook, you will:

- Understand the functionality of the `ocrqa` subpackage from the Impresso Pipelines package.
- Learn how to calculate a **QA score** for a given raw text.
- Discover how the pipeline **automatically detects language** using impresso [`langident`](https://github.com/impresso/impresso-datalab-notebooks/tree/main/annotate/langident_pipeline_demo.ipynb) before applying QA scoring.
- Explore different use cases, including **basic and advanced usage** of the OCR QA pipeline.
- Recognize some limitations of the pipeline, such as handling **uncommon words and short texts**.

By the end of this notebook, you will have a clear understanding of how the OCR QA score is computed and its practical applications.


---

## Prerequisites


First, you should install Impresso package:


In [ ]:
%pip install impresso_pipelines[ocrqa]

---

## Workflow


### Basic Usage

Start by importing the necessary module from Impresso Pipelines package


In [ ]:
from impresso_pipelines.ocrqa import OCRQAPipeline

ocrqa_pipeline = OCRQAPipeline()

Once you initialize the pipeline, you can simply provide the text you'd like to classify. This example demonstrates the use of German text.


In [ ]:
de_text = "Ein kleiner Hund wqeg Max lebte in einem ruhigen Dorf."

In [ ]:
ocrqa_pipeline(de_text)

If no language is explicitly specified, the OCR QA pipeline uses the Impresso
`langident` pipeline to automatically detect the language of the text. For more details
on the `langident` pipeline, please refer to the [langident
pipeline demo notebook](https://github.com/impresso/impresso-datalab-notebooks/tree/main/annotate/langident_pipeline_demo.ipynb).

Once the language is detected, the pipeline checks if a corresponding Bloom filter exists. The default output of the pipeline is a dictionary containing the detected language and the corresponding QA score. The computed QA score is a rough measure and is intentionally rounded to one decimal place to account for minor variations, such as the presence of unusual names or OCR errors, which should not significantly impact the overall score.

In this example, the score is **0.9**, meaning that approximately **one out of ten words** does not exist in the Bloom filter. This suggests that while the OCR process was largely successful, there was a small misidentification (_wqeg_) in the text.


### Advanced Usage


This pipeline offers several additional attributes that can be used when calling it to gain a deeper understanding of the results. These attributes include `language`, `version`, `diagnostics`, `model_id`, and `supported_languages`:

- `language`: Accepts language abbreviation strings such as "en" (English) or "de" (German). If provided, the pipeline assumes the specified language and skips the language detection step, directly using the corresponding Bloom filter.

- `version`: Accepts a specific Bloom filter model version in the format "1.0.5" or "1.0.6". If specified, the pipeline uses the requested version (if available) and skips the automatic retrieval of the latest model.

- `diagnostics`: Boolean. If set to True, the pipeline returns additional information, such as known_tokens, unknown_tokens, and the Bloom filter name used. For more details, see the sections below.

- `model_id`: Boolean. If set to True, the pipeline includes the name of the Bloom filter model used in the output.

- `supported_languages`: Boolean. If set to True, the pipeline returns a list of supported languages (i.e., languages for which a Bloom filter is available).

These attributes can be used individually, in combination with each other, or all at once, depending on the level of detail needed.


**Example 1**: `language`


In [ ]:
# Using the same German text example as before
ocrqa_pipeline(de_text, language="lb")

Even though the provided text is clearly in German, specifying the language as Luxembourgish, for example, forces the pipeline to use the corresponding Bloom filter for that language. If the selected language is unsupported, the pipeline will return an appropriate error message.


**Example 2**: `version`


In [ ]:
# Using the same German text example as before
ocrqa_pipeline(de_text, version="1.0.5")

In the example above, by explicitly setting the `version` to _1.0.5_ , you are instructing the pipeline to use the Bloom filter corresponding to this version, even if a more recent version is available.


**Example 3**: `diagnostics`


In [ ]:
# Using the same German text example as before
ocrqa_pipeline(de_text, diagnostics=True)

Once you set `diagnostics` to _True_ , an additional key, `diagnostics`, will be added to the dictionary. The value of this key contains all known and unknown tokens, as well as the name of the Bloom filter used. In this example, we can see that there are no unknown words, meaning every word exists in this specific Bloom filter.


**Example 4**: `model_id`


In [ ]:
# Using the same German text example as before
ocrqa_pipeline(de_text, model_id=True)

Similar to the `diagnostics` attribute, the `model_id` attribute is a simpler version. If set to `True`, the pipeline will return an additional key, `bloom_filter`, with the value indicating the Bloom filter that was used for the analysis.


**Example 5**: `supported languages`


In [ ]:
# Using the same German text example as before
ocrqa_pipeline(de_text, supported_languages=True)

Once `supported_languages` is set to _True_, the pipeline returns an additional key, `supported_languages`, with a value containing a list of all currently supported languages (i.e., languages that have a corresponding Bloom filter).


**Example 6**: All at once


In [ ]:
# Using the same German text example as before
ocrqa_pipeline(
    de_text,
    language="fr",
    version="1.0.5",
    diagnostics=True,
    model_id=True,
    supported_languages=True,
)

You can use a mix of additional parameters, or all of them at once, to gain a deeper understanding of your QA score. In the example above, we set the language to French, which results in many unknown tokens being identified, as the Bloom filter used may not cover certain French words.


### Limitations of the OCR QA Score Pipeline


In [ ]:
short_de_text_with_unusual_name = "Glebs geht ins Büro."

In [ ]:
# Example 1: Very short sentence
ocrqa_pipeline(short_de_text_with_unusual_name, diagnostics=True)

As shown, the language detection model struggles with **short texts**, leading to potential misclassification. Additionally, **uncommon names** or **rare words** can lower the QA score, even if they were correctly identified by OCR. This happens because these words do not exist in the current version of the Bloom filter, making them appear as unknown tokens.


---

## Conclusion


### Summary

In this notebook, we've explored the OCR QA Pipeline from the Impresso package, which evaluates OCR quality by calculating a score between 0 and 1. The pipeline:

- Automatically detects text language using [`langident`](https://github.com/impresso/impresso-datalab-notebooks/tree/main/annotate/langident_pipeline_demo.ipynb) (or accepts manual language specification)
- Uses Bloom filters to identify known/unknown words and calculate a quality score
- Offers additional parameters for customization (`language`, `version`, `diagnostics`, `model_id`, `supported_languages`)
- Has limitations with short texts and uncommon words
- Provides a standardized approach to OCR quality assessment across multiple languages

The pipeline is particularly useful for evaluating OCR quality in historical documents and identifying potential errors in digitized text.


### Next steps


Since this subpackage relies on the Impresso langident subpackage, you might be interested in exploring the [demo notebook](https://github.com/impresso/impresso-datalab-notebooks/tree/main/annotate/langident_pipeline_demo.ipynb) for langident.

Additionally, you can find the repository for the [**Impresso Pipelines package**](https://github.com/impresso/impresso-pipelines/tree/mallet_pipeline).


---
## Project and License info

### Impresso project

[Impresso - Media Monitoring of the Past](https://impresso-project.ch) is an interdisciplinary research project that aims to develop and consolidate tools for processing and exploring large collections of media archives across modalities, time, languages and national borders. The first project (2017-2021) was funded by the Swiss National Science Foundation under grant No. [CRSII5_173719](http://p3.snf.ch/project-173719) and the second project (2023-2027) by the SNSF under grant No. [CRSII5_213585](https://data.snf.ch/grants/grant/213585) and the Luxembourg National Research Fund under grant No. 17498891.

### Copyright

Copyright (C) 2024 The Impresso team.

### License

This program is provided as open source under the [GNU Affero General Public License](https://github.com/impresso/impresso-pyindexation/blob/master/LICENSE) v3 or later.

---

<p align="center">
  <img src="https://github.com/impresso/impresso.github.io/blob/master/assets/images/3x1--Yellow-Impresso-Black-on-White--transparent.png?raw=true" width="350" alt="Impresso Project Logo"/>
</p>
